**왓챠피디아(https://peida.watcha.com/ko-KR) 리뷰 분석**

**과제 옵션**
1. 제공된 데이터셋/본인이 직접 선택한 데이터셋을 통해 간단한 CBOW/skip-gram 모델 훈련해보기  
2. t-sne를 활용해 직접 훈련한 word2vec 모델에 다양한 단어를 입력해 시각화해보기  
3. 직접 훈련한 word2vec 모델에 관심있는 단어를 입력하고 해당 단어와 유사한 단어들 살펴보기  
4. 특정 두 단어 사이의 유사도(거리)를 측정해보고 모델이 잘 훈련되었는지 판단해보기  
5. word2vec 모델 훈련 시 window_size 등 다양한 초모수 설정 변경해가며 결과 비교해보기  
6. gensim 패키지에서 제공하는 사전학습 모델을 활용해 각자 창의적으로 해보고 싶은 것 해보기

### 1. 전처리

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

movie_review = pd.read_csv("./왓챠피디아 기묘한 이야기 시즌1~4 코멘트.csv",index_col = 0)
movie_review[:5]

,코멘트
0,아이들에 대한 묘사가 너무 절묘하다. 유난히 똑똑한 아이들임을 보여주면서도 순진함 ...
1,"스토리, 음악, 캐릭터까지 '슈퍼 에이트'보다 더 7080스럽고 사랑스럽지만 동시에..."
2,왜 아무도 바브에게 관심을 쏟지 않는 거죠...
3,재밌을랑말랑하는 기묘함
4,취향은 아니지만 보는것을 멈출수가없었네요..


In [4]:
# 결측값 존재 유무 확인
print(movie_review.isnull().values.any())

False


In [5]:
# 리뷰 개수 출력
print(len(movie_review))

922


In [274]:
# 정규 표현식 이용, 한글 외 문자 제거
movie_review['코멘트'] = movie_review['코멘트'].str.replace("[^ ㄱ-ㅎㅏ-ㅣ가-힣]","")
movie_review[:5]

,코멘트
0,아이들에 대한 묘사가 너무 절묘하다 유난히 똑똑한 아이들임을 보여주면서도 순진함 속...
1,스토리 음악 캐릭터까지 슈퍼 에이트보다 더 스럽고 사랑스럽지만 동시에 중독성 강한 ...
2,왜 아무도 바브에게 관심을 쏟지 않는 거죠
3,재밌을랑말랑하는 기묘함
4,취향은 아니지만 보는것을 멈출수가없었네요


### 띄어쓰기 교정

In [254]:
import sys
sys.path.append('..')
from pykospacing.kospacing import Spacing
spacing = Spacing()

In [275]:
new_sent = []
for sent in movie_review['코멘트']:
    new_sent.append(spacing(sent))

In [276]:
new_corpus = new_sent
movie_review['코멘트_space'] = new_corpus

### 네이버 한글 맞춤법 검사기 : 띄어쓰기 포함되어 있음!

In [261]:
from hanspell import spell_checker

In [277]:
spelled_sent = []
for sent in movie_review['코멘트']:
    spell_sent = spell_checker.check(sent)
    checked_sent = spell_sent.checked
    spelled_sent.append(checked_sent)
        
movie_review['코멘트_spell_checker'] = spelled_sent
movie_review

,코멘트,코멘트_space,코멘트_spell_checker
0,아이들에 대한 묘사가 너무 절묘하다 유난히 똑똑한 아이들임을 보여주면서도 순진함 속...,아이들에 대한 묘사가 너무 절묘하다 유난히 똑똑한 아이들임을 보여주면서도 순진함 속...,아이들에 대한 묘사가 너무 절묘하다 유난히 똑똑한 아이들임을 보여주면서도 순진함 속...
1,스토리 음악 캐릭터까지 슈퍼 에이트보다 더 스럽고 사랑스럽지만 동시에 중독성 강한 ...,스토리 음악 캐릭터까지 슈퍼 에이트보다 더 스럽고 사랑스럽지만 동시에 중독성 강한 ...,스토리 음악 캐릭터까지 슈퍼 에이트보다 더스럽고 사랑스럽지만 동시에 중독성 강한 호...
2,왜 아무도 바브에게 관심을 쏟지 않는 거죠,왜 아무도 바브에게 관심을 쏟지 않는 거죠,왜 아무도 바브에게 관심을 쏟지 않는 거죠
3,재밌을랑말랑하는 기묘함,재밌을 랑 말랑하는 기 묘함,재밌을랑 말랑하는 기묘함
4,취향은 아니지만 보는것을 멈출수가없었네요,취향은 아니지만 보는 것을 멈출 수가 없었네요,취향은 아니지만 보는 것을 멈출 수가 없었네요
...,...,...,...
917,ㄹㄹㄹ,ㄹㄹㄹ,ㄹㄹㄹ
918,,,
919,기묘한가 재미있는가 의문을 가지고 보지만 답은 역시 명성대로,기묘한 가 재미있는 가 의문을 가지고 보지만 답은 역시 명성대로,기묘한가 재미있는가 의문을 가지고 보지만 답은 역시 명성대로
920,미쳐,미쳐,미쳐


In [278]:
del movie_review['코멘트_space']
movie_review.columns = ["코멘트_orig","코멘트_clean"]

In [379]:
# 불용어 정의
stopwords = ['이','있','하','것','들','그','되','수','보','않','없','나','사람','주','아니','등','같','우리',
            '때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','두','말하','알','그러나',
            '받','모사','그런','똣','문제','더','사회','많','그리고','좋','크','따르','중','나오','가지','씨',
            '시키','만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신','안','어떤',
            '내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번','나','다른','어떻','여자','개',
            '전','들','사실','이렇','점','싶','말','정도','좀','원','잘','통하','소리','높','다','없다','않다',
            '의','과','이다','를','있다','도','에','을','와','왜','고','같다','하다','으로','가다','는','보다',
            '인','로','너무','에서','되다','게','은','화','거','자다','까지','싶다','못','하고','들다','그렇다',
            '정말','만','력','주다','에는']


# 형태소 분석기 OKT를 사용한 토큰화 작업
okt = Okt()
tokenized_data = []
for sentence in movie_review['코멘트_clean']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

### 2. 모델 훈련(CBOW vs. skip-gram)

In [380]:
from gensim.models import Word2Vec

In [381]:
# CBOW 방법 사용
model1 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 100, 
                 min_count = 5,
                 window = 5, 
                 sg = 0)

In [382]:
# skip-gram 방법 사용
model2 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 100, 
                 min_count = 5,
                 window = 5, 
                 sg = 1)

**Parameters**  
```workers``` : 실행할 병렬 프로세스의 수, 코어수, 주로 4-6사이 지정

```size``` : 각 단어에 대한 임베딩 된 벡터차원 정의, size=2라면 한 문장의 벡터는 [-0.1248574, 0.255778]와 같은 형태를 가지게 된다.

```min_count``` : 단어에 대한 최소 빈도수. min_count=5라면 빈도수 5 이하 무시

```window``` : 문맥 윈도우 수, 양쪽으로 몇 개의 단어까지 고려해서 의미를 파악할 것인지 지정하는 것

```sample``` : 빠른 학습을 위해 정답 단어 라벨에 대한 다운샘플링 비율을 지정하는 것, 보통 0.001이 좋은 성능을 낸다고 한다.

```sg``` : 1이면 skip-gram 방법을 사용하고, 0이면 CBOW 방법을 사용한다. → default = 1

```iter``` : epoch와 같은 뜻으로 학습 반복 횟수를 지정한다.

In [368]:
# CBOW 완성된 임베딩 매트릭스의 크기 확인
print(model.wv.vectors.shape)

# skip-gram 모델 완성된 임베딩 매트릭스의 크기 확인
print(model2.wv.vectors.shape)

(694, 100)
(692, 100)


#### ```wv.most_similar()``` : 가장 유사한 단어들을 추출할 수 있다.

In [346]:
model.wv.most_similar("스토리") # CBOW

[('시즌', 0.9943159818649292),
 ('드라마', 0.9938040971755981),
 ('재밌다', 0.9934017658233643),
 ('연기', 0.9933466911315918),
 ('아이', 0.9932005405426025),
 ('연출', 0.9931702613830566),
 ('캐릭터', 0.9927265048027039),
 ('많다', 0.9927006959915161),
 ('일레븐', 0.9925738573074341),
 ('좋다', 0.9925539493560791)]

In [369]:
model2.wv.most_similar("스토리") # skip-gram

[('흥미진진', 0.9996491074562073),
 ('미치다', 0.9996362924575806),
 ('연출', 0.9996293187141418),
 ('일레븐', 0.9996097087860107),
 ('귀엽다', 0.9995805621147156),
 ('많다', 0.9995736479759216),
 ('볼', 0.9995663166046143),
 ('버리다', 0.9995652437210083),
 ('추천', 0.9995623230934143),
 ('등장인물', 0.999561071395874)]

In [342]:
model.wv.most_similar("호러") # CBOW

[('좋다', 0.9910829663276672),
 ('드라마', 0.9909542798995972),
 ('연기', 0.9905955791473389),
 ('시즌', 0.9904861450195312),
 ('아이', 0.9904405474662781),
 ('느낌', 0.9904223680496216),
 ('나오다', 0.9904062747955322),
 ('괴물', 0.9896777868270874),
 ('캐릭터', 0.989277720451355),
 ('아니다', 0.9891306161880493)]

In [383]:
model2.wv.most_similar("호러") #skip-gram

[('괴물', 0.9995923042297363),
 ('세계', 0.9995922446250916),
 ('시절', 0.9995921850204468),
 ('느낌', 0.9995903968811035),
 ('미스터리', 0.99958336353302),
 ('스릴러', 0.9995710253715515),
 ('느끼다', 0.9995632171630859),
 ('주인공', 0.9995575547218323),
 ('살다', 0.9995497465133667),
 ('굉장하다', 0.999549388885498)]

#### ```wv.similarity()``` : 유비(analogy), 두 단어를 입력해 코사인 유사도 계산

In [395]:
model2.wv.similarity('호러','스릴러')

0.9995711

In [462]:
model2.wv.similarity('윌','조이스') #부정확

0.99920076

## 2. 초모수 값 변경

### 1) size 모수 조절 : 50~300
- 각 단어에 대한 임베딩 된 벡터차원 정의
- 너무 작은 값이 아니라면 학습의 경향이 달라지지 않는다.  
- 보통 논문/실험 등에서는 100~300 정도의 값으로 설정  
- size가 클수록 더 큰 메모리가 필요하지만 더 정확한 모델을 만들 수 있음  
- 적절한 차원 수 계산 = 단어의 개수 ** 0.25  
⇒ 출처 : https://stackoverflow.com/questions/48479915/what-is-the-preferred-ratio-between-the-vocabulary-size-and-embedding-dimension

#### size = 200

In [477]:
model3 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 200, 
                 min_count = 5,
                 window = 5, 
                 sg = 1)

model3.wv.most_similar("스토리") # skip-gram

[('미치다', 0.9997945427894592),
 ('귀엽다', 0.9997934103012085),
 ('추천', 0.9997921586036682),
 ('흥미진진', 0.9997838735580444),
 ('궁금하다', 0.9997825622558594),
 ('많다', 0.9997814893722534),
 ('일레븐', 0.9997813105583191),
 ('걸', 0.9997766613960266),
 ('긴장감', 0.9997761249542236),
 ('등장인물', 0.9997757077217102)]

In [478]:
model3.wv.most_similar("호러") #skip-gram

[('스릴러', 0.9997791051864624),
 ('미스터리', 0.9997729063034058),
 ('괴물', 0.9997543692588806),
 ('판타지', 0.999752938747406),
 ('연대', 0.9997494220733643),
 ('살다', 0.9997481107711792),
 ('영화', 0.9997472763061523),
 ('세계', 0.9997472763061523),
 ('느끼다', 0.9997464418411255),
 ('장르', 0.9997435808181763)]

#### size = 300

In [475]:
model4 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 300, 
                 min_count = 5,
                 window = 5, 
                 sg = 1)

model4.wv.most_similar("스토리") # skip-gram

[('대단하다', 0.9998520016670227),
 ('긴장감', 0.9998484253883362),
 ('보여주다', 0.9998483061790466),
 ('일레븐', 0.9998472929000854),
 ('걸', 0.9998466372489929),
 ('시리즈', 0.999846339225769),
 ('미치다', 0.9998456835746765),
 ('느끼다', 0.999845027923584),
 ('배우', 0.9998441934585571),
 ('모든', 0.9998440146446228)]

In [476]:
model4.wv.most_similar("호러") #skip-gram

[('스릴러', 0.9998499751091003),
 ('미스터리', 0.9998387098312378),
 ('연대', 0.9998334646224976),
 ('판타지', 0.9998325109481812),
 ('성장', 0.9998301863670349),
 ('느끼다', 0.9998301267623901),
 ('영화', 0.9998282790184021),
 ('드라마', 0.9998273849487305),
 ('장르', 0.9998252391815186),
 ('향수', 0.9998241662979126)]

#### size = 10

In [481]:
model5 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 10, 
                 min_count = 5,
                 window = 5, 
                 sg = 1)

model5.wv.most_similar("스토리") # skip-gram

[('기다리다', 0.9989835023880005),
 ('미치다', 0.998934805393219),
 ('와중', 0.9988172650337219),
 ('재현', 0.9987524747848511),
 ('존나', 0.9987190961837769),
 ('무섭다', 0.9987044334411621),
 ('시리즈', 0.9986265897750854),
 ('가득하다', 0.9986243844032288),
 ('감동', 0.9986122250556946),
 ('흥미진진', 0.9985907673835754)]

In [482]:
model5.wv.most_similar("호러") #skip-gram

[('시절', 0.9988580346107483),
 ('스릴러', 0.9987044334411621),
 ('킹', 0.9986584186553955),
 ('긴장', 0.9986554980278015),
 ('판', 0.9985576868057251),
 ('크리처', 0.9985381364822388),
 ('시키다', 0.9984337091445923),
 ('아쉽다', 0.9983357191085815),
 ('맛', 0.9983145594596863),
 ('판타지', 0.9982568621635437)]

### 2) min_count 모수 조절 : 2~100
- 최소 빈도수  
- 해당 빈도수보다 작게 등장한 단어는 모델 학습에서 배제  
- 값을 작게 할수록 빈도 적은 단어들도 계산에 포함돼 속도가 느려지고 모델의 크기가 커짐  
- 일반적으로 10 ~ 100 사이 값 권장

#### min_count = 10

In [488]:
model2_1 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 200, 
                 min_count = 10,
                 window = 5, 
                 sg = 1)

model2_1.wv.most_similar("스토리") # skip-gram

[('미치다', 0.9997718334197998),
 ('추천', 0.9997693300247192),
 ('귀엽다', 0.9997686743736267),
 ('연출', 0.9997653961181641),
 ('대단하다', 0.9997615814208984),
 ('궁금하다', 0.9997596144676208),
 ('일레븐', 0.9997562170028687),
 ('긴장감', 0.9997526407241821),
 ('흥미진진', 0.9997520446777344),
 ('배우', 0.9997481107711792)]

In [489]:
model2_1.wv.most_similar("호러") # skip-gram

[('스릴러', 0.9997598528862),
 ('미스터리', 0.9997431039810181),
 ('괴물', 0.999737024307251),
 ('느끼다', 0.999732494354248),
 ('나오다', 0.999730110168457),
 ('판타지', 0.9997291564941406),
 ('연기', 0.9997258186340332),
 ('장르', 0.9997247457504272),
 ('세계', 0.9997239708900452),
 ('오다', 0.9997239708900452)]

#### min_count = 50

In [494]:
model2_2 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 200, 
                 min_count = 50,
                 window = 5, 
                 sg = 1)

model2_2.wv.most_similar("스토리") # skip-gram

[('재밌다', 0.20830407738685608),
 ('기묘하다', 0.20131590962409973),
 ('시즌', 0.18763238191604614),
 ('드라마', 0.18192651867866516),
 ('애', 0.18060463666915894),
 ('나오다', 0.14819592237472534),
 ('연대', 0.12232963740825653),
 ('이야기', 0.0951845794916153),
 ('캐릭터', 0.07883840799331665),
 ('느낌', 0.06769584119319916)]

In [495]:
model2_2.wv.most_similar("호러") # skip-gram

KeyError: "word '호러' not in vocabulary"

### 3) window 모수 조절 : 2~10
- 타깃 단어를 예측하기 위해 사용할 앞뒤 맥락 단어의 개수  
- 값이 커지면 단어의 학습량이 증가 → 계산량 증가, 단어의 의미적 정확도 증가  
- 값이 클수록 주요 도메인 정보를 더 많이 잡아낼 수 있음  
- 값이 작을수록 단어 하나하나 자체에 대해 더 많이 잡아낼 수 있음

#### window = 1

In [2]:
model3_1 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 200, 
                 min_count = 5,
                 window = 1, 
                 sg = 1)

model3_1.wv.most_similar("스토리") # skip-gram

NameError: name 'Word2Vec' is not defined

In [1]:
model3_1.wv.most_similar("호러") # skip-gram

NameError: name 'model3_1' is not defined

#### window = 3

In [498]:
model3_2 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 200, 
                 min_count = 5,
                 window = 3, 
                 sg = 1)

model3_2.wv.most_similar("스토리") # skip-gram

[('드라마', 0.9998314380645752),
 ('재밌다', 0.9998151063919067),
 ('캐릭터', 0.9998118877410889),
 ('시즌', 0.9998093843460083),
 ('많다', 0.9998037219047546),
 ('귀엽다', 0.9998030662536621),
 ('괴물', 0.9998029470443726),
 ('연출', 0.9997992515563965),
 ('연기', 0.9997958540916443),
 ('일레븐', 0.9997954368591309)]

In [499]:
model3_2.wv.most_similar("호러") # skip-gram

[('나오다', 0.9997599720954895),
 ('드라마', 0.9997506141662598),
 ('캐릭터', 0.9997338652610779),
 ('연기', 0.9997336864471436),
 ('괴물', 0.9997214078903198),
 ('이야기', 0.9997164011001587),
 ('스토리', 0.9997121691703796),
 ('느끼다', 0.9997121095657349),
 ('영화', 0.9997093081474304),
 ('연대', 0.9997087717056274)]

#### window = 10

In [500]:
model3_3 = Word2Vec(sentences = tokenized_data,
                 workers = 4,
                 size = 200, 
                 min_count = 5,
                 window = 10, 
                 sg = 1)

model3_3.wv.most_similar("스토리") # skip-gram

[('긴장감', 0.999727189540863),
 ('뛰어나다', 0.9997236728668213),
 ('선사', 0.9997175931930542),
 ('연출', 0.9997164607048035),
 ('등등', 0.9997159242630005),
 ('액션', 0.9997130632400513),
 ('대다', 0.9997124671936035),
 ('너무나', 0.9997103214263916),
 ('훌륭하다', 0.999708890914917),
 ('밖에', 0.9997082948684692)]

In [502]:
model3_3.wv.most_similar("호러") # skip-gram

[('스릴러', 0.9997023344039917),
 ('미스터리', 0.9997010827064514),
 ('장르', 0.9996613264083862),
 ('가지다', 0.999659538269043),
 ('연대', 0.9996564388275146),
 ('섞다', 0.9996458292007446),
 ('판타지', 0.9996434450149536),
 ('온갖', 0.9996329545974731),
 ('스러운', 0.9996294975280762),
 ('크리처', 0.9996261596679688)]